In [2]:
from ScrapeFB import *
import pandas as pd
import bs4, time, re, json
from datetime import datetime, timedelta

In [4]:
window = WindowNoLogin(url='https://www.facebook.com/FreeYOUTHth/')

In [5]:
window.scrape(filename='freeyouth.json')

100%|██████████| 10/10 [01:53<00:00, 11.39s/it]


KeyboardInterrupt: 

In [ ]:
window.scrape(filename='freeyouth.json')